In [32]:
import pypyodbc

server = '192.168.0.21'
port = '4900'
database = 'BAC_A_SABLE'
username = 'admin'
password = 'Clip_SERENA'


cnxn = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

crsr = cnxn.cursor()

table_name = 'CHARGES'
#cf_name = 'C6800'
semaine = '12'

#query = 'SELECT * FROM {}'.format(table_name)
#query = 'SELECT * FROM ' +table_name+ ' WHERE NAF < 1000'
query = "SELECT COSECT FROM " +table_name+ " WHERE SEMAINE = " +semaine
#COFRAIS like '" +cf_name+ "' AND

records = crsr.execute(query).fetchall()
print(records)

#columns_list = list(crsr.columns())
#columns_list

[('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATMGB',), ('ATMGB',), ('ATMGB',), ('ATMGB',), ('ATMGB',), ('ATMGB',), ('ATMGB',), ('ATMGB',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATLBT',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATCRM',), ('ATLBT',), ('ATLBT',)]


#### one_row = crsr.fetchone()
type(one_row)